# Import Libraries

In [1]:
# Imports

import pandas as pd
import numpy as np
import csv

# Load Datasets

In [2]:
# Load NBER categories
f_name = "nber.tsv"
dtypes = {'category_id': np.int8, 'subcategory_id': np.int8}
nber   = pd.read_csv(f_name, delimiter="\t", dtype=dtypes, quoting=csv.QUOTE_NONNUMERIC)

In [3]:
# Load patent data
f_name = "patent.tsv"
dtypes = {'num_claims': np.int16, 'withdrawn': np.float32}
patent = pd.read_csv(f_name, delimiter="\t", dtype=dtypes, parse_dates = ['date'])

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0,2,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Clean Datasets

In [4]:
# Get relevant columns
nber = nber[['patent_id','subcategory_id']]
patent = patent[['id', 'type', 'date']]

# Adjust data types to ensure proper matching
nber = nber.astype({'patent_id': 'str'})
patent = patent.astype({'id': 'str'})

# Rename patent 'id' column to 'patent_id'
patent = patent.rename(columns={"id":"patent_id"})

In [5]:
# Create year column based on date
patent['year'] = patent['date'].astype(str).str[:4]

# Merge Datasets

In [6]:
# Merge patent and nber on patent_id
patentNber = patent.merge(nber, on="patent_id", how='inner')\
        .rename(columns={"subcategory_id": "patent_subcategory_id"})

In [7]:
# Filter for only utility patents
patentNber = patentNber[patentNber['type'] == 'utility']

# Aggregate

In [8]:
# Get subcategories
list_subcat = sorted(patentNber['patent_subcategory_id'].dropna().unique())

In [9]:
# Dataframe for storing aggregate data
df_patentNber = pd.DataFrame()

# Iterate through the subcategories
for patent_subcategory_id in list_subcat:
    # Generate one row for each year
    temp = patentNber[patentNber['patent_subcategory_id'] == patent_subcategory_id].groupby("year").count()\
        [['patent_subcategory_id']].rename(columns={'patent_subcategory_id': patent_subcategory_id})

    # Merge the temp column into the result matrix
    df_patentNber = df_patentNber.merge(temp, left_index=True, right_index=True, how = "outer")


In [10]:
# Replace NaN with 0
df_patentNber = df_patentNber.replace(np.nan,0)
# Visualize the data
df_patentNber.style.applymap(lambda x: 'background-color : yellow' if x > 0 else '').format('{:2}')

In [11]:
# Get patents created in the US
f_name   = "location.tsv"
location = pd.read_csv(f_name, delimiter="\t", quoting=csv.QUOTE_NONNUMERIC)
f_name   = "patent_inventor.tsv"
inventors = pd.read_csv(f_name, delimiter="\t", quoting=csv.QUOTE_NONNUMERIC)

In [14]:
# Filter only those location ids with US
us_locations = location[location['country'] == 'US']
# Create list of these location ids
us_locations = list(us_locations['id'])

In [15]:
# Filter inventors dataframe for only those that match us location ids
us_inventors = inventors[inventors['location_id'].isin(us_locations)]

In [16]:
# If multiple inventors for the same patent from the US, there will be duplicates
grouped_inventors = us_inventors.drop_duplicates(subset=['patent_id'])

In [18]:
# Merge the datasets
inventors_patents = patentNber.merge(grouped_inventors, on='patent_id')

,patent_id,type,date,year,patent_subcategory_id,inventor_id,location_id
0,3930273,utility,1976-01-06,1976,65,3930273-1,926f6cfd-7bf1-426d-b7bd-8fad5f57842b
1,3930274,utility,1976-01-06,1976,55,3930274-1,88007623-d869-4abc-9baf-c9cc0c08f4ff
2,3930275,utility,1976-01-06,1976,63,3930275-1,f7a99802-8ee9-4f5d-a58c-0fc5b6e18a3c
3,3930276,utility,1976-01-06,1976,69,3930276-1,4a350101-c754-45df-8c3c-21be86ed01ff
4,3930277,utility,1976-01-06,1976,69,3930277-1,fb6d9122-5864-4086-ad95-dec18b1c4f56
...,...,...,...,...,...,...,...
2721154,RE31607,utility,1984-06-19,1984,21,4130816-2,7108f7bd-1552-4ed0-9173-9e8956fa7d11
2721155,RE31697,utility,1984-10-09,1984,45,4229942-3,bc3d3acf-22d3-4cfd-a03a-b7dc68d9c2bb
2721156,RE31699,utility,1984-10-09,1984,43,4082017-1,902d1119-ba54-4453-9fbb-d4cf2c5f3e13
2721157,RE31704,utility,1984-10-09,1984,45,3973165-1,c9a23860-3b85-4af1-aa5a-d15b9edbc9ff


In [19]:
list_subcat = sorted(inventors_patents['patent_subcategory_id'].dropna().unique())

In [20]:
# Dataframe for storing aggregate data
usonly_df_patentNber = pd.DataFrame()

# Iterate through the subcategories
for patent_subcategory_id in list_subcat:
    # Generate one row for each year
    temp = inventors_patents[inventors_patents['patent_subcategory_id'] == patent_subcategory_id].groupby("year").count()\
        [['patent_subcategory_id']].rename(columns={'patent_subcategory_id': patent_subcategory_id})

    # Merge the temp column into the result matrix
    usonly_df_patentNber = usonly_df_patentNber.merge(temp, left_index=True, right_index=True, how = "outer")


In [21]:
# Replace NaN with 0
usonly_df_patentNber = usonly_df_patentNber.replace(np.nan,0)
# Visualize the data
usonly_df_patentNber.style.applymap(lambda x: 'background-color : yellow' if x > 0 else '').format('{:2}')

In [24]:
# Export the data
df_patentNber.to_csv('patents_by_year.csv')
usonly_df_patentNber.to_csv('usonly_patents_by_year.csv')